<a href="https://colab.research.google.com/github/dumingyu/Academic-project-page-template/blob/master/mental_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.21.0 datasets evaluate
!wget https://raw.githubusercontent.com/dumingyu/EATD-Corpus-dataset/main/all.csv
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: requests
    F

In [ ]:
# 'GBK' to 'UTF-8'
with open('all.csv', 'r', encoding='GBK') as file:
    content = file.read()

with open('all.csv', 'w', encoding='UTF-8') as file:
    file.write(content)

# load
dataset = load_dataset('csv', data_files='all.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Filter:   0%|          | 0/324 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 324
    })
})

In [ ]:
datasets = dataset["train"].train_test_split(0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 259
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 65
    })
})

In [ ]:
#model_name = "hfl/chinese-roberta-wwm-ext"
#model_name = "google-bert/bert-base-chinese"
#model_name = "google-bert/bert-base-multilingual-cased"
#model_name = "google-bert/bert-base-uncased"
model_name = "FacebookAI/roberta-base"
#model_name = "FacebookAI/xlm-roberta-base"
#model_name = "FacebookAI/xlm-roberta-large"
#model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, num_labels=2)
tokenizer

PreTrainedTokenizerFast(name_or_path='FacebookAI/roberta-base', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [ ]:
def process_function(examples):
  tokenized_examples = tokenizer(examples["review"], max_length=64, truncation=True)
  tokenized_examples["labels"] = examples["label"]
  return tokenized_examples

In [ ]:
tokenized_datasets = datasets.map(process_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/259 [00:00<?, ? examples/s]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'review', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 259
    })
    test: Dataset({
        features: ['label', 'review', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 65
    })
})

In [ ]:
accuracy_metric = evaluate.load("accuracy")
accuracy_metric

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = predictions.argmax(axis=-1)
  return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
args = TrainingArguments(
  learning_rate=2e-5,
  per_device_train_batch_size=32,
  per_device_eval_batch_size=128,
  num_train_epochs=5,
  weight_decay=0.01,
  output_dir="model_for_seqclassification",
  logging_steps=10,
  evaluation_strategy = "epoch",
  save_strategy = "epoch",
  load_best_model_at_end=True,
  metric_for_best_model="accuracy",
  fp16=True,
)

Some weights of the model checkpoint at FacebookAI/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.out

In [ ]:
trainer = Trainer(
  model,
  args=args,
  train_dataset=tokenized_datasets["train"],
  eval_dataset=tokenized_datasets["test"],
  tokenizer=tokenizer,
  compute_metrics=compute_metrics,
  data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

trainer.train()
trainer.evaluate()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: review. If review are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 259
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 45


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.703531,0.384615
2,0.694300,0.704064,0.384615
3,0.696500,0.686869,0.492308
4,0.677400,0.668273,0.600000
5,0.627000,0.648627,0.646154


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: review. If review are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 65
  Batch size = 128
Saving model checkpoint to model_for_seqclassification/checkpoint-9
Configuration saved in model_for_seqclassification/checkpoint-9/config.json
Model weights saved in model_for_seqclassification/checkpoint-9/pytorch_model.bin
tokenizer config file saved in model_for_seqclassification/checkpoint-9/tokenizer_config.json
Special tokens file saved in model_for_seqclassification/checkpoint-9/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: review. If review are not expected by `RobertaForSequenceClassification.forward`,  you can safely i

{'eval_loss': 0.6486272215843201,
 'eval_accuracy': 0.6461538461538462,
 'eval_runtime': 0.1084,
 'eval_samples_per_second': 599.355,
 'eval_steps_per_second': 9.221,
 'epoch': 5.0}